In [1]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import matplotlib.pyplot as plot
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime

In [2]:
file = "./demographic_NY/census_block_demographic_NY.geojson"  
block_pop = gp.read_file(file)
block_pop.head()

,STATEFP10,GEOID,B02001e1,B02001e2,B02001e3,B02001e4,B02001e5,B02001e6,B03002e12,geometry
0,36,15000US360070121011,607,600,7,0,0,0,0,"MULTIPOLYGON (((-75.89054 42.18697, -75.89920 ..."
1,36,15000US360070121014,954,935,0,0,19,0,0,"MULTIPOLYGON (((-75.87509 42.17416, -75.87541 ..."
2,36,15000US360070121013,1186,1142,44,0,0,0,0,"MULTIPOLYGON (((-75.83838 42.19741, -75.83813 ..."
3,36,15000US360070121024,1074,968,0,0,69,0,23,"MULTIPOLYGON (((-75.85079 42.16052, -75.85114 ..."
4,36,15000US360070121023,823,714,0,0,0,0,0,"MULTIPOLYGON (((-75.87457 42.17416, -75.87338 ..."


In [3]:
#Rename columns
block_pop.rename(columns = {"B02001e6":"hawaiian"}, inplace = True)
block_pop.rename(columns = {"B03002e12":"hispanic_latino"}, inplace = True)
block_pop.rename(columns = {"B02001e5":"asian"}, inplace = True)
block_pop.rename(columns = {"B02001e2":"white"}, inplace = True)
block_pop.rename(columns = {"B02001e3":"black"}, inplace = True)
block_pop.rename(columns = {"B02001e1":"total"}, inplace = True)
block_pop.rename(columns = {"B02001e4":"indian_alaska"}, inplace = True)

#Drop column
#df = df.drop (columns = ['ALAND10'])
block_pop.head()

,STATEFP10,GEOID,total,white,black,indian_alaska,asian,hawaiian,hispanic_latino,geometry
0,36,15000US360070121011,607,600,7,0,0,0,0,"MULTIPOLYGON (((-75.89054 42.18697, -75.89920 ..."
1,36,15000US360070121014,954,935,0,0,19,0,0,"MULTIPOLYGON (((-75.87509 42.17416, -75.87541 ..."
2,36,15000US360070121013,1186,1142,44,0,0,0,0,"MULTIPOLYGON (((-75.83838 42.19741, -75.83813 ..."
3,36,15000US360070121024,1074,968,0,0,69,0,23,"MULTIPOLYGON (((-75.85079 42.16052, -75.85114 ..."
4,36,15000US360070121023,823,714,0,0,0,0,0,"MULTIPOLYGON (((-75.87457 42.17416, -75.87338 ..."


In [5]:
file = "./NY/NY.geojson"  
precincts = gp.read_file(file)
precincts.head()

,id,canonical,type,centerX,centerY,neighbors,geometry
0,36001263,NY Albany 263,ORIGINAL,+42.7272097,-073.7873353,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,NY Albany 232,ORIGINAL,+42.7295842,-073.7227157,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.71119 ..."
2,36001342,NY Albany 342,ORIGINAL,+42.7348777,-073.7093285,"36001233,36001243,36001340,36001341,36001339,3...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,NY Albany 341,ORIGINAL,+42.7355901,-073.7037615,"36001233,36001342,36001339,36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,NY Albany 277,ORIGINAL,+42.7313271,-073.8385576,"36001275,36001276,36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [6]:
precinct_demographics = precincts[['id', 'geometry']].copy()

#Add type columns
precinct_demographics.insert(1, 'total', 0)
precinct_demographics.insert(2, 'white', 0)
precinct_demographics.insert(3, 'black', 0)
precinct_demographics.insert(4, 'hispanic_latino', 0)
precinct_demographics.insert(5, 'asian', 0)
precinct_demographics.insert(6, 'indian_alaska', 0)
precinct_demographics.insert(7, 'hawaiian', 0)

precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian,geometry
0,36001263,0,0,0,0,0,0,0,"MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,0,0,0,0,0,0,0,"MULTIPOLYGON (((-73.71364 42.72799, -73.71119 ..."
2,36001342,0,0,0,0,0,0,0,"MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,0,0,0,0,0,0,0,"MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,0,0,0,0,0,0,0,"MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [7]:
#Data assign based if census block or precinct enclosed
for i_precinct, row_precinct in precinct_demographics.iterrows():
    if i_precinct % 1000 == 0:
            print (datetime.datetime.now().strftime("%H:%M:%S"), i_precinct)    
    for i_block, row_block in block_pop.iterrows():
        
        if row_precinct.geometry.contains(row_block.geometry) or row_precinct.geometry.within(row_block.geometry):
            print ("Precinct", row_precinct['id'], 'contains block ', i_block)
            precinct_demographics.at[i_precinct, 'total'] += row_block['total']
            precinct_demographics.at[i_precinct, 'white'] += row_block['white']
            precinct_demographics.at[i_precinct, 'black'] += row_block['black']
            precinct_demographics.at[i_precinct, 'hispanic_latino'] += row_block['hispanic_latino']
            precinct_demographics.at[i_precinct, 'asian'] += row_block['asian']
            precinct_demographics.at[i_precinct, 'indian_alaska'] += row_block['indian_alaska']
            precinct_demographics.at[i_precinct, 'hawaiian'] += row_block['hawaiian']
            block_pop = block_pop.drop(i_block)
    

22:10:02 0
Precinct 3600141 contains block  3758
Precinct 3600120 contains block  3757
Precinct 36005492 contains block  14822
Precinct 3600528 contains block  14262
Precinct 36005296 contains block  13980
Precinct 36005608 contains block  14987
Precinct 36005453 contains block  15194
Precinct 36005442 contains block  14751
Precinct 36005423 contains block  14834
Precinct 36005213 contains block  13715
Precinct 36005243 contains block  14410
Precinct 36005609 contains block  14633
Precinct 36005609 contains block  14636
Precinct 36005609 contains block  14637
Precinct 36005195 contains block  15251
Precinct 36005770 contains block  13798
Precinct 36005796 contains block  14024
Precinct 36005901 contains block  14716
Precinct 36005112 contains block  14155
Precinct 36005112 contains block  14157
22:42:36 1000
Precinct 36005773 contains block  14494
Precinct 36005773 contains block  14608
Precinct 36005816 contains block  14029
Precinct 36005919 contains block  14491
Precinct 36005542 co

Precinct 36119827 contains block  6677
Precinct 36119108 contains block  8094
Precinct 36119831 contains block  6820
Precinct 36119623 contains block  7333
Precinct 36119452 contains block  7003
Precinct 36119937 contains block  7485
Precinct 36119259 contains block  7824


In [8]:
#Data assign based on how much census block area inside of a precinct 
for i_precinct, row_precinct in precincts.iterrows():
    if i_precinct % 1000 == 0:
            print (datetime.datetime.now().strftime("%H:%M:%S"), i_precinct)
            
    for i_block, row_block in block_pop.iterrows():
        intersection = row_precinct.geometry.intersection(row_block.geometry)
        if intersection.area > 0:
            ratio = intersection.area / row_precinct.geometry.area
            precinct_demographics.at[i_precinct, 'total'] += round(row_block['total'] * ratio)
            precinct_demographics.at[i_precinct, 'white'] += round(row_block['white'] * ratio)
            precinct_demographics.at[i_precinct, 'black'] += round(row_block['black'] * ratio)
            precinct_demographics.at[i_precinct, 'hispanic_latino'] += round(row_block['hispanic_latino'] * ratio)
            precinct_demographics.at[i_precinct, 'asian'] += round(row_block['asian'] * ratio)
            precinct_demographics.at[i_precinct, 'indian_alaska'] += round(row_block['indian_alaska'] * ratio)
            precinct_demographics.at[i_precinct, 'hawaiian'] += round(row_block['hawaiian'] * ratio)

06:13:38 0
06:44:39 1000
07:15:52 2000
07:46:56 3000
08:17:59 4000
08:48:54 5000
09:19:56 6000
09:50:57 7000
10:21:51 8000
10:52:50 9000
11:23:57 10000
11:55:49 11000
12:28:33 12000
13:01:01 13000
13:32:15 14000


In [9]:
precinct_demographics = precinct_demographics.drop(columns = ['geometry'])
pd.DataFrame(precinct_demographics).to_json("./demographic_NY/precinct_demographic_NY.json")

In [11]:
precinct_demographics.head(10)

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian
0,36001263,2301,2077,102,59,57,6,0
1,36001232,1435,1157,96,75,171,0,0
2,36001342,759,599,97,29,5,1,0
3,36001341,485,467,2,5,0,0,0
4,36001277,1706,1447,36,169,141,0,0
5,36001346,2089,1533,233,161,46,13,0
6,36001278,1177,1076,43,19,33,4,0
7,36001251,2726,2471,21,51,185,3,0
8,36001339,692,595,92,6,0,0,0
9,36001340,736,559,147,20,0,0,0


In [4]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [3]:
precinct_demographics = pd.read_json("./demographic_NY/precinct_demographic_NY.json")
precinct_demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian
0,36001263,2301,2077,102,59,57,6,0
1,36001232,1435,1157,96,75,171,0,0
2,36001342,759,599,97,29,5,1,0
3,36001341,485,467,2,5,0,0,0
4,36001277,1706,1447,36,169,141,0,0


In [5]:
add_demographic_querry = ("INSERT INTO mhaczynska.DEMOGRAPHIC_DATA "
               "(PRECINCT_GEOID, STATE_NAME, TOTAL_POPULATION, WHITE, "
               " BLACK, HISPANIC_LATINO, ASIAN, INDIAN_ALASKAN, HAWAIIAN_PACIFIC) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")

#update_precinct_querry = "UPDATE mhaczynska.precinct SET CENTER_X = %s, CENTER_Y = %s WHERE GEO_ID = %s"

STATE = 'NY'
for index, row in precinct_demographics.iterrows(): 
    demographic_data = (row['id'].item(), STATE, row['total'].item(), row['white'].item(), row['black'].item() , 
                        row['hispanic_latino'].item(), row['asian'].item(), row['indian_alaska'].item(), row['hawaiian'].item())
    #update_precinct = (row['centerX'], row['centerY'], row['id'])
    cursor.execute(add_demographic_querry, demographic_data)
    
    mydb.commit()
    if index == 500:
        break

In [7]:
cursor.close()
mydb.close()

In [7]:
### ANOMALOUS DATA
file = "./demographic_NY/precinct_demographic_NY.json"
demographics = pd.read_json(file)
demographics.head()

,id,total,white,black,hispanic_latino,asian,indian_alaska,hawaiian
0,36001263,2301,2077,102,59,57,6,0
1,36001232,1435,1157,96,75,171,0,0
2,36001342,759,599,97,29,5,1,0
3,36001341,485,467,2,5,0,0,0
4,36001277,1706,1447,36,169,141,0,0


In [8]:
empty_df = block_pop[block_pop['B02001e1'] == 0]
empty_df.shape

(309, 10)

In [9]:
file = "./NY/NY_final.geojson"  
precincts = gp.read_file(file)
precincts.head()

,id,canonical,type,centerX,centerY,neighbors,geometry
0,36001263,NY Albany 263,ORIGINAL,+42.7272097,-073.7873353,"36001262,36001253,36001247,36001252,36001265,3...","MULTIPOLYGON (((-73.79031 42.72261, -73.79523 ..."
1,36001232,NY Albany 232,ORIGINAL,+42.7295842,-073.7227157,"36001243,36001249,36001343,36001344,36001251,3...","MULTIPOLYGON (((-73.71364 42.72799, -73.70998 ..."
2,36001342,NY Albany 342,ORIGINAL,+42.7348777,-073.7093285,"36001233,36001243,36001340,36001341,36001339,3...","MULTIPOLYGON (((-73.71210 42.73705, -73.71182 ..."
3,36001341,NY Albany 341,ORIGINAL,+42.7355901,-073.7037615,"36001233,36001342,36001339,36001285","MULTIPOLYGON (((-73.70277 42.73421, -73.70857 ..."
4,36001277,NY Albany 277,ORIGINAL,+42.7313271,-073.8385576,"36001275,36001276,36001273","MULTIPOLYGON (((-73.82521 42.72768, -73.82606 ..."


In [11]:
count = 0
for i, row in precincts.iterrows():
    for j, block in empty_df.iterrows():
        intersection = row.geometry.intersection(block.geometry)
        if intersection.area > 0:
            ratio = intersection.area / block.geometry.area
            if ratio > 0.90:
                print(ratio, row['id'], block['GEOID'])
                count += 1
                #demographics.at[i, 'total'] = 0
                #demographics.at[i, 'white'] = 0
                #demographics.at[i, 'hispanic_latino'] = 0
                #demographics.at[i, 'asian'] = 0
                #demographics.at[i, 'indian_alaska'] = 0
                #demographics.at[i, 'hawaiian'] = 0
print(count)

0.9939813830400663 360038889 15000US360039402001
0.9979809956801348 36005650 15000US360050504000
0.911242078023989 36005650 15000US360050504001
0.9999981087261802 36005530 15000US360050293010
0.9729634043313723 36005177 15000US360050171001
0.9702511707302723 3600537 15000US360050210012
0.9988280690947694 36005548 15000US360050138000
0.9864320484112508 36005548 15000US360050160000
0.9897222900180115 36005882 15000US360050028000
0.9153208460576991 36005661 15000US360050276000
0.9343124942691292 36005859 15000US360050024001
0.9959199082036316 36005606 15000US360050516000
0.9999997988473092 36005606 15000US360050516005
0.9932678123265454 36005937 15000US360050249001
0.9861169823167448 36005551 15000US360050110000
0.9848775060787519 36005926 15000US360050117000
0.9916599230700806 36005926 15000US360050002000
0.9993629839037051 36005926 15000US360050024000
0.906535598856649 36005836 15000US360050117002
0.9998790458720244 36005553 15000US360050132000
0.90597306391592 36005785 15000US360050041

0.9887443744578724 36081118 15000US360810934010
0.9973240208578936 36081111 15000US360810916021
0.9984660299169638 36081367 15000US360810997050
0.9998283033938449 36081367 15000US360810999001
0.9766906850175902 36081355 15000US360810973000
0.9838305245179035 36081355 15000US360810981000
0.9909596008530319 36081355 15000US360810987000
0.9842394835603027 36081458 15000US360810939000
0.9990526199371801 36081453 15000US360810991000
0.9928949993561148 360811346 15000US360810107010
0.9962911341405422 36081420 15000US360811483000
0.9999111995652433 36081712 15000US360810624001
0.9283888783170302 360811181 15000US360810331001
0.9900710404150497 360811360 15000US360810099001
0.9996000914603259 36081406 15000US360811093000
0.9957083172093943 3608514 15000US360850018000
0.9999727454704003 36085211 15000US360850176000
0.9851455313551282 36085258 15000US360850248000
0.9973740665127384 36085256 15000US360850244010
0.9775530161344602 36085114 15000US360850007000
0.9999992539299353 36085114 15000US360